In [6]:
print("Importing packages...")
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import numpy as np
import torch
import json
import os
from sklearn.model_selection import train_test_split


Importing packages...


In [9]:
EPOCHS = 80
START_EPOCH = 0
LEARNING_RATE = 2e-4
LEARNING_RATE_STR = "2e-4"
train_file = "Data/roberta_trainset.csv"
test_file = "Data/roberta_testset.csv"
val_file = "Data/roberta_validation.csv"
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
val = pd.read_csv(val_file)
train_labels = [[int(y) for y in x.replace("[","").replace("]","").split(", ")] for x in train.labels]
test_labels = [[int(y) for y in x.replace("[","").replace("]","").split(", ")] for x in test.labels]
val_labels = [[int(y) for y in x.replace("[","").replace("]","").split(", ")] for x in val.labels]
train.labels = train_labels
test.labels = test_labels
val.labels = val_labels

parameters = {'train_batch_size':10, 
                    'gradient_accumulation_steps':16, 
                    'learning_rate': LEARNING_RATE, 
                    'num_train_epochs': 1, 
                    'max_seq_length': 512, 
                    'output_dir': f'ROBERTA_OUTPUTS/Learning_rate_{LEARNING_RATE_STR}/current/', 
                    'overwrite_output_dir':True}

for epoch in range(START_EPOCH, EPOCHS):
    if(epoch == 0):
        print("Creating RoBERTa model")
        model = MultiLabelClassificationModel('roberta', 'roberta-base',num_labels=len(train_labels[0]), args = parameters)
    else:
        print("Importing model saved at previous epoch...")
        torch.cuda.empty_cache()
        model = MultiLabelClassificationModel('roberta', f'ROBERTA_OUTPUTS/Learning_rate_{LEARNING_RATE_STR}/epoch{epoch - 1}',
                                          args = parameters)
    print("Training...")
    model.train_model(train)
    if(epoch != 0): 
        old_val_loss = new_val_loss
    else:
        old_val_loss = 999
    print("Validating...")
    result, model_outputs, wrong_predictions = model.eval_model(val)
    new_val_loss = result['eval_loss']
    print(f"New validation loss = {new_val_loss}, old validation loss = {old_val_loss}")
    if(old_eval_loss < new_val_loss):
        print("Validation loss increased, stopping training...")
        break
    else:
        print("Validation loss keeps going down, proceeding with next epoch")
    print(f"Saving model for epoch {epoch}/{EPOCHS}...")
    os.system(f"cp -r ROBERTA_OUTPUTS/Learning_rate_{LEARNING_RATE_STR}/current ROBERTA_OUTPUTS/Learning_rate_{LEARNING_RATE_STR}/epoch{epoch}")

print("Importing best model...")
torch.cuda.empty_cache()
model = MultiLabelClassificationModel('roberta', f'ROBERTA_OUTPUTS/Learning_rate_{LEARNING_RATE_STR}/epoch{epoch - 1}',
                                          args = parameters)

print("Testing best model...")
_, test_outputs = model.predict(test['text'])
test_outputs = pd.DataFrame(test_outputs)

print("Saving test results")
test_outputs.to_csv(f"Results/RoBERTa_results/Learning_rate_{LEARNING_RATE_STR}/test_outputs_epoch{epoch - 1}.csv")

Creating RoBERTa model
Training...



New validation loss = 0.6969316126062319, old validation loss = 999


NameError: name 'old_eval_loss' is not defined